# RT2 Assignment 2 - Jupyter
### Maria Luisa Aiachini - 4375373

This code represents the user interface node.
Using this file the user is able to control the robot by:
- Start and stop random behavior using two buttons
- Controlling linear and angular velocities of the robot using two sliders
- Controlling directly the direction of movement of the robot using a keypad

Also the user can check the stats of the robot with:
- A line plot that shows the actual linear and angular velocities vs the requested ones
- A bar plot showing the number of reached and canceled goals
- A XY graph showing the current position of the robot
- A histogram showing the time needed to reach every target

## How to run the simulation
At first you need to clone [this repository](https://github.com/Marilwoo/rt2_assignment2) in a ros workspace. Build the package in a ros1 sourced terminal running
```
catkin_make
```
For running the simulation the user needs to open two terminals, both sourced with ROS.
In the first one the simulation will be run, as well as all the nodes but this one.
Run:
```
roslaunch rt2_assignment2 sim.launch
```
In the second terminal, move in the folder 
```
ros_ws/src/rt2_assignment2/notebook
```
Here run:
```
jupyter notebook --allow-root
```
Once the jupyter is opened  go in the file `user_interface.ipynb` and run every node.
At this point everything is running and the user is able to decide how to interact with the robot.


## Importing needed libraries, defining publishers and clients

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import rospy
import numpy as np
import ipywidgets as widgets
import math
import time
import tf
import jupyros as jr
import actionlib
import actionlib.msg
import rt2_assignment1.msg
from matplotlib.animation import FuncAnimation
from matplotlib import animation, rc
from IPython.display import display
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox, Label, interact, interactive, fixed, interact_manual
from rt2_assignment1.srv import Command
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
from actionlib_msgs.msg import GoalID
from std_msgs.msg import Bool, Duration
from tf.transformations import quaternion_matrix

%matplotlib widget

# Declaring global variables
reached = 0
cancelled = 0
counter = 0
linear_array = []
y = [0,0]
pos_x = []
pos_y = []
tempo = []

# Init of the node and declaring client for the user interface and the publisher for the cmd_vel to 
# define velocities
rospy.init_node('user_interface')
ui_client = rospy.ServiceProxy('/user_interface', Command)
pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1 )

## Random behavior buttons
Defining the two buttons for the random navigation. The first one is for starting the robot and the second one is for stopping immediately the robot.

In [2]:
# Definitions of the two buttons for the random behaviour and their layout
b1 = widgets.Button(description='Start random behaviour',
        layout=Layout(width='auto', align="center", grid_area='b1'),
        style=ButtonStyle(button_color='palegreen'))

b2 = widgets.Button(description='Stop random behaviour',
        layout=Layout(width='auto', grid_area='b2'),
        style=ButtonStyle(button_color='salmon'))

# Callback functions for the two buttons for the random behavour
def start_rand(b):
    ui_client('start')

    
def stop_rand(b):
    ui_client('stop')

b1.on_click(start_rand)
b2.on_click(stop_rand)

HBox([b1, b2])



## Velocity sliders
Defining the two sliders for changing manually the linear and angular velocity of the robot.

In [3]:
# Defining the two sliders and their values
slider1 = widgets.FloatSlider(description = 'linear vel', min = 0.0, max = 3.0, step = 0.2, value = 0.0)
slider2 = widgets.FloatSlider(description = 'angular vel', min = 0.0, max = 3.0, step = 0.2, value = 0.0)
output = widgets.Output()
display(slider1, slider2, output)

# Defining the callbacks for the movement of the two sliders
def set_linear(change):
    with output:
        twist_msg = Twist()
        twist_msg.linear.x = slider1.value
        pub.publish(twist_msg)

        
def set_angular(change):
    with output:
        twist_msg = Twist()
        twist_msg.angular.z = slider2.value
        pub.publish(twist_msg)

slider1.observe(set_linear, names='value')
slider2.observe(set_angular, names='value')

FloatSlider(value=0.0, description='linear vel', max=3.0, step=0.2)

FloatSlider(value=0.0, description='angular vel', max=3.0, step=0.2)

Output()

## Buttons for controlling manually
Defining the 5 buttons for moving manually the robot. Four buttons for moving and one for stopping.

In [4]:
# Defining the five buttons for manually moving the robot
b1 = widgets.Button(description='Left',
        layout=Layout(width='auto', align = 'center', grid_area = 'b1'),
        style=ButtonStyle(button_color='palegreen'))

b2 = widgets.Button(description='Right',
        layout=Layout(width='auto', align = 'center', grid_area = 'b2'),
        style=ButtonStyle(button_color='palegreen'))

b3 = widgets.Button(description='Straight',
        layout=Layout(width='auto', align = 'center', grid_area = 'b3'),
        style=ButtonStyle(button_color='palegreen'))

b4 = widgets.Button(description='Back',
        layout=Layout(width='auto', align = 'center', grid_area = 'b4'),
        style=ButtonStyle(button_color='palegreen'))

b5 = widgets.Button(description='STOP',
        layout=Layout(width='auto', align = 'center', grid_area = 'b5'),
        style=ButtonStyle(button_color='salmon'))

# Defining the position of the five buttons
GridBox(children=[b1, b2, b3, b4, b5],
    layout=Layout(
    width='50%',
    grid_template_rows='30px 30px 30px',
    grid_template_columns='20% 20% 20%',
    grid_template_areas='''
" . b3 . "
"b1 b4 b2 "
"b5 b5 b5"
''')
    )

GridBox(children=(Button(description='Left', layout=Layout(grid_area='b1', width='auto'), style=ButtonStyle(bu…

In [5]:
# Callback functions for the buttons of the manual mode
def straight(b):
    twist_msg = Twist()
    ui_client('stop')
    twist_msg.linear.x = 0.5
    twist_msg.angular.z = 0
    time.sleep(1)
    pub.publish(twist_msg)
    
def back(b):
    twist_msg = Twist()
    ui_client('stop')
    twist_msg.linear.x = -0.5
    twist_msg.angular.z = 0
    time.sleep(1)
    pub.publish(twist_msg)
    
def right(b):
    twist_msg = Twist()
    ui_client('stop')
    twist_msg.linear.x = 0
    twist_msg.angular.z = 0.8
    time.sleep(1)
    pub.publish(twist_msg)
    
def left(b):
    twist_msg = Twist()
    ui_client('stop')
    twist_msg.linear.x = 0
    twist_msg.angular.z = -0.8
    time.sleep(1)
    pub.publish(twist_msg)
    
def stop(b):
    twist_msg = Twist()
    ui_client('stop')
    twist_msg.linear.x = 0
    twist_msg.angular.z = 0
    time.sleep(1)
    pub.publish(twist_msg)
    
b1.on_click(left)
b2.on_click(right)
b3.on_click(straight)
b4.on_click(back)
b5.on_click(stop)

## Line plot: velocities
Plotting the real velocity and the requested one, both linear and angular velocity.

In [6]:
start = time.time() # Starting the timer for the progression of the plot
time1 = list(range(0, 10))
fig1, ax1 = plt.subplots()

# Defining the four lines for the plot
line1, = ax1.plot([], [], color='k', label='actual velocity') 
line2, = ax1.plot([], [], color='r', label='requested velocity')
line3, = ax1.plot([], [], color='b', label='actual angular velocity')
line4, = ax1.plot([], [], color='y', label='requested angular velocity')

#Setting labels, title and positions of the legend
ax1.set_xlabel('Time') 
ax1.set_ylabel('Velocity')
ax1.set_title("Actual velocities and requested velocities")
plt.legend(loc='upper right')
y_data1=[]
y_data2=[]
y_data3=[]
y_data4=[]

def init1():
    line1.set_data([], [])
    line2.set_data([],[])
    line3.set_data([], [])
    line4.set_data([],[])
    return [line1, line2, line3, line4]

# Callback for the /odom subscriber
def vel_callback(msg):
    global pos_x, pos_y
    if len(time1)>50:
        time1.pop(0)
     
    if len(y_data1)>50:
        y_data1.pop(0)
    if len(y_data3)>50:
        y_data3.pop(0)
    y_data1.append(msg.twist.twist.linear.x)
    y_data3.append(msg.twist.twist.angular.x) 
    time1.append(time.time()-start)
    #Taking from the  /odom topic also the position for the last plot
    pos_x.append(msg.pose.pose.position.x)
    pos_y.append(msg.pose.pose.position.y)
    
    
sub = rospy.Subscriber('/odom', Odometry, vel_callback)

# Callback for the /cmd_vel subscriber
def cmd_vel_callback(msg):   
    if len(y_data2)>50:
        y_data2.pop(0)
    if len(y_data4)>50:
        y_data4.pop(0)
    y_data2.append(msg.linear.x)
    y_data4.append(msg.angular.z)

sub_vel = rospy.Subscriber('/cmd_vel', Twist, cmd_vel_callback)


# Function for updating the plot in real time
def animate1(i):
    line1.set_data(time1, y_data1)
    line2.set_data(time1, y_data2)
    line3.set_data(time1, y_data3)
    line4.set_data(time1, y_data4)
    ax1.axis([time1[0]+2,time1[9]+0.2,-3,3])   
    return [line1,line2, line3, line4]
anim = animation.FuncAnimation(fig1, animate1, init_func=init1,
                               frames=100, interval=20, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Bar plot: reached and canceled goals
Plotting using a bar plot the number of reached targets and the number of canceled goals.

In [7]:
# Callback for the /reached subscriber
def reach_callback(msg):
    global y, reached, cancelled
    if msg.data == True:  
        reached = reached + 1
    else:
        cancelled = cancelled + 1
    y = [reached, cancelled]


sub2 = rospy.Subscriber('/reached', Bool, reach_callback)

# Defining the figure
x = ['reached', 'cancelled']
fig2, ax2 = plt.subplots()
ax2.bar(x,y, color = 'b', width = 0.)
ax2.set_title('Reached and canceled goals')

def init2():
    ax2.bar(x,y, color = 'b', width = 0.5)

# Function for updating the figure in real time
def animate2(i):
    global x, y, ax2
    ax2.bar(x,y, color = 'b', width = 0.5)
    
ani2=animation.FuncAnimation(fig2, animate2, init_func=init2, frames=100, interval=100, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## XY graph:position of the robot
Plotting an xy graph containing the current position of the robot at each time.

In [8]:
# Defining the plot, labels and title
fig4, ax4 = plt.subplots()
plt.plot([], [], 'ro')
line5,= ax4.plot([],[])
ax4.set_xlim(-5, 5)
ax4.set_ylim(-5, 5)
ax4.set_xlabel('X')
ax4.set_ylabel('Y')
ax4.set_title('Current X and Y robot position')

# Initialization for the plot
def plot_init():
    line5.set_data([],[])

# Function for updating the plot in real time
def update_plot(frame):
    global pos_x, pos_y
    line5.set_data(pos_x, pos_y)
    return line5

ani = FuncAnimation(fig4, update_plot, init_func=plot_init, frames = 100, interval = 100, blit = True)
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Hist plot: time required for reaching each target
Plotting the time required for reaching each target using a histogram.

In [9]:
def time_callback(msg):
    global tempo
    tempo.append(msg.data.secs)

subbb = rospy.Subscriber('/duration', Duration, time_callback)


fig5, ax5 = plt.subplots()
ax5.hist(tempo, bins = 20, color= 'blue', align = 'mid', stacked='true', edgecolor = 'w')
ax5.set_xlim(10, 50)
ax5.set_xlabel('Time (seconds)')
ax5.set_title('Time required to reach each target')

# Initialization for the plot
def plot_init5():
    ax5.hist(tempo,bins = 20, color= 'blue', align = 'mid', stacked='true', edgecolor = 'w')

# Function for updating the plot in real time
def update_plot5(frame):
    global tempo
    ax5.hist(tempo,bins = 20, color= 'blue', align = 'mid', stacked='true', edgecolor = 'w')

ani5 = FuncAnimation(fig5, update_plot5, init_func=plot_init5, frames = 100, interval = 100, blit = True)
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …